## Macbookでも意外にLLMを動くことができる

最近、OpenAI O1 レベルのOSSモデルDeepSeek R1がでました。それで、XではM4 64GBのMac miniを8台繋いてクラスターにして671Bのモデルを動いたスレットがありました。

<blockquote class="twitter-tweet" data-media-max-width="560">

<p lang="en" dir="ltr">

Running DeepSeek-V3 on M4 Mac Mini AI Cluster<br><br>671B MoE model distributed across 8 M4 Pro 64GB Mac Minis.<br><br>Apple Silicon with unified memory is a great fit for MoE. <a href="https://t.co/FmeARutaxq">pic.twitter.com/FmeARutaxq</a>

</p>

— EXO Labs (@exolabs) <a href="https://twitter.com/exolabs/status/1872444906851229814?ref_src=twsrc%5Etfw">December 27, 2024</a>

</blockquote>


```{=html}
<script async src="https://platform.twitter.com/widgets.js" charset="utf-8"></script>
```


スレットを見た時に、「嘘らだろう、671BのモデルをMac miniで動く？」は最初の感想でした。色々調べた後、LLMを動く際には、制限としては「計算スピード」と「メモリのスピード」の2つあることがわかりました。Batch sizeが1で推論する場合は、だいたい計算スピードではなく、メモリのスピードで足が引っ張られています。

例えば、`int4` で量子化したモデルで推論する場合、A100とMac mini M4を比較して見たら、

|    デバイス     | メモリスピード |    計算力 |
|:---------------:|---------------:|----------:|
|      A100       |      1935 GB/s | 1248 TOPS |
| Mac mini M4 |       120 GB/s |   38 TOPS |
|  A100/ Macmini  |             16x |       32x |

A100の計算力はMac miniの32倍ですが、メモリスピードは7倍しかないです。なので理論上はMac miniはA100の1/7のスピードで出力することができます。

これからは、この話を更に展開して原理までわかるようにします。

## GPUの仕組みを理解する

CPU の構造とよく似ていて、GPUもキャッシュがあります。下図でメモリのアーキテクチャを示しています。

![](images/paste-4.png){width="456"}

-   **GPU SRAM：** GPUの計算ユニット内蔵のメモリ。最速だが、最も容量が小さい（19TB/s、20MB）。

-   **GPU HBM：** GPUのメインメモリ。中間の速度と容量（1.5TB/s、40GB）。

-   **メインメモリ DRAM：** 最も低速だが、最も容量が大きい（12.8GB/s、\>1TB）。

計算する前に、まずモデルのパラメータをCPUのメモリからGPU HBMに送る必要があります。`model.to("cuda")`はこれをやっています。計算する際に、必要なデータをHBMからGPUチップに内装されるSRAMに転送し、そこで計算をしています。計算が終わっていても、次のデータがまだ来ていない場合は、計算を止めて、データの転送を待たないと行けないことです。バッチサイズが小さい場合は、計算量が少ないため、データの転送を待つことになり、これはいわゆる「メモリバンド」のことです。

## 実際に推論スピードを概算してみる

### 概算方法

まず、概算するための式を出します。

$$
\text{latency}_\text{memory} := \dfrac{P\cdot n_{\text{bytes}}}{n_{\text{memory bandwidth}}}
$$

$$
\text{latency}_\text{compute} := \dfrac{2 \cdot P \cdot B}{n_{\text{flops}}}
$$

この中で、

-   $P$はモデルのパラメータ数。

-   $n_{\text{bytes}}$はデータタイプに必要なバイト数。例えば、デフォルトのfp32を使う場合は4バイト、fp16の場合は2バイト、int4の場合は0.5バイト。

-   $n_{\text{memory bandwidth}}$は名前の通り、メモリ帯域幅のこと。

-   $B$はバッチサイズ。

-   $n_{\text{flops}}$は計算スピード

メモリのレイテンシーは割とわかりやすいです。分子は1トークンを計算するために計算ユニットのメモリ(SRAM)に送るデータ量のことです。それをメモリ帯域幅に割ると、データ転送の時間を概算することができます。

計算のレイテンシーだと少しややこしいです。概算する際には、経験則で1トークンにかかる計算量を$2P$とします(この後で詳細に計算してみます)。それをかけるバッチサイズ$B$にすると、$B$個のトークンを計算するために必要な計算量になります。それを計算スピードで割ると、計算の時間を概算することができます。


### A100とMac mini M4を比較する

これで、計算スピードとメモリスピードの比較ができるようになりました。表にある内容を式に代入して、A100とMac mini M4を実際に比較して比較してみます。

|    デバイス     | メモリ観点で<br>1秒処理できるトークン数 |    計算力観点で<br>1秒処理できるトークン数 |
|:---------------:|---------------:|----------:|
|      A100       |      552 | 89,142 |
| Mac mini M4 |       34 |   2,714 |
|  A100/ Macmini  |             16x |       32x |


In [ ]:
#| code-fold: true
P = 7e9  # 7Bモデル
n_bytes = 0.5  # int4
n_memory_bandwidth_A100 = 1935e9
n_memory_bandwidth_Macmini = 120e9

n_tops_A100 = 1248e12
n_tops_Macmini = 38e12


def memory_latency(n_bytes, n_memory_bandwidth, P):
    return P * n_bytes / n_memory_bandwidth


def compute_latency(n_tops, P, B=1):
    return 2 * P * B / n_tops


memory_latency_A100 = memory_latency(n_bytes, n_memory_bandwidth_A100, P)
memory_latency_Macmini = memory_latency(n_bytes, n_memory_bandwidth_Macmini, P)

compute_latency_A100 = compute_latency(n_tops_A100, P)
compute_latency_Macmini = compute_latency(n_tops_Macmini, P)

print('1/memory_bandwidth_A100: ', int(1 / memory_latency_A100))
print('1/memory_bandwidth_Macmini: ', int(1 / memory_latency_Macmini))
print('1/compute_A100: ', int(1 / compute_latency_A100))
print('1/compute_Macmini: ', int(1 / compute_latency_Macmini))
print('A100/Macmini memory: ', int(memory_latency_Macmini / memory_latency_A100))
print('A100/Macmini compute: ', int(compute_latency_Macmini / compute_latency_A100))